In [ ]:
import numpy as np
import pandas as pd
import os, sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def train_data():
    data = utils.DBtable_to_df('../integration.duckdb', 'integratedTable')

    X = data[data.columns[1:].tolist()] # features
    y = data['Madrid_section'] # target

    X_train = pd.read_pickle('../Feature generation/data-X_train.pkl.bz2', compression='bz2')
    y_train = pd.read_pickle('../Feature generation/data-y_train.pkl.bz2', compression='bz2')
    X_test  = pd.read_pickle('../Feature generation/data-X_test.pkl.bz2', compression='bz2')
    y_test  = pd.read_pickle('../Feature generation/data-y_test.pkl.bz2', compression='bz2')

    return X, y, X_train, X_test, y_train, y_test

gridSearch = True

# Building Random Forest

In [ ]:
X, y, X_train, X_test, y_train, y_test = train_data()

if gridSearch:
    n_est = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_d = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_d.append(None)
    parameters = {'n_estimators': n_est, 'max_features': ['auto','sqrt'], 'bootstrap': [True,False], 'max_depth': max_d, 'min_samples_split': [2,5,10], 'min_samples_leaf': [1, 2, 4]}
    
else :
    parameters = {'n_estimators': [889], 'min_samples_split': [10], 'min_samples_leaf': [2], 'max_features': ['sqrt'], 'max_depth': [800], 'bootstrap': [True]} 

print(parameters)
model = RandomForestClassifier()
grid = RandomizedSearchCV(estimator = model, param_distributions = parameters, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_
test_acc = best_model.score(X=X_test, y=y_test)
train_acc = best_model.score(X=X_train, y=y_train)

print('\nRandom Forest:')
print('- Best Params:', grid.best_params_)
print(f'- Accuracy: {{ train: {train_acc:.3f}, CV: {grid.best_score_:.3f}, test: {test_acc:.3f}}}')

Model metrics (validation)

In [ ]:
print(classification_report(y_test,grid.predict(X_test),target_names=['NoDef', 'Def'], digits=3))

cm = confusion_matrix(y_test, grid.predict(X_test))
print ('\nConfusion matrix: \n', cm)